🔴 Avancé | ⏱ 45 min | 🔑 Concepts : coverage.py, pytest-cov, tox, nox

# 04 - Couverture de Code et Automatisation des Tests

## Objectifs

- Mesurer la **couverture de code** avec coverage.py et pytest-cov
- Interpréter les rapports de couverture
- Comprendre la différence entre couverture de lignes et de branches
- Automatiser les tests avec **tox** et **nox**
- Créer un workflow de tests professionnel

## Prérequis

- Maîtrise de pytest (voir notebook 03)
- Connaissance de base de Python
- Familiarité avec les environnements virtuels

## 1. Qu'est-ce que la couverture de code ?

La **couverture de code** (code coverage) mesure le pourcentage de code exécuté par vos tests.

### Types de couverture

| Type | Description | Exemple |
|------|-------------|----------|
| **Line coverage** | % de lignes exécutées | `if x > 0:` → ligne couverte |
| **Branch coverage** | % de branches (if/else) testées | `if/else` → les deux branches testées ? |
| **Function coverage** | % de fonctions appelées | Fonction jamais appelée = 0% |
| **Statement coverage** | % d'instructions exécutées | Similaire à line coverage |

### Pourquoi mesurer la couverture ?

- ✅ Identifier le code **non testé**
- ✅ Éviter les **régressions** dans le code non couvert
- ✅ **Confiance** dans la suite de tests
- ❌ 100% de couverture ≠ code sans bugs

## 2. Coverage.py : L'outil de base

In [ ]:
# Installation
!pip install coverage pytest-cov -q

In [ ]:
%%writefile math_utils.py
"""Module mathématique pour démonstration de couverture."""

def absolute(x):
    """Retourne la valeur absolue de x."""
    if x < 0:
        return -x
    else:
        return x

def max_de_trois(a, b, c):
    """Retourne le maximum de trois nombres."""
    if a >= b and a >= c:
        return a
    elif b >= a and b >= c:
        return b
    else:
        return c

def factorielle(n):
    """Calcule la factorielle de n."""
    if n < 0:
        raise ValueError("n doit être positif")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

def diviser_ou_zero(a, b):
    """Divise a par b, retourne 0 si b == 0."""
    if b == 0:
        return 0
    return a / b

In [ ]:
%%writefile test_math_utils.py
"""Tests (incomplets) pour math_utils."""
import pytest
from math_utils import absolute, max_de_trois, factorielle, diviser_ou_zero

def test_absolute_positif():
    """Test seulement les nombres positifs."""
    assert absolute(5) == 5
    assert absolute(0) == 0
    # Manque : test avec nombre négatif !

def test_max_de_trois_simple():
    """Test basique."""
    assert max_de_trois(1, 2, 3) == 3
    # Manque : tests des autres branches !

def test_factorielle():
    """Test de la factorielle."""
    assert factorielle(0) == 1
    assert factorielle(5) == 120
    # Manque : test de l'exception !

# Fonction diviser_ou_zero jamais testée !

### Exécuter les tests avec couverture

In [ ]:
# Méthode 1 : coverage run + coverage report
!coverage run -m pytest test_math_utils.py
!coverage report -m

## 3. pytest-cov : Intégration avec pytest

pytest-cov rend l'utilisation de coverage plus simple.

In [ ]:
# Exécuter pytest avec couverture
!pytest test_math_utils.py --cov=math_utils --cov-report=term-missing

In [ ]:
# Générer un rapport HTML interactif
!pytest test_math_utils.py --cov=math_utils --cov-report=html
print("\nRapport HTML généré dans htmlcov/index.html")

### Options utiles de pytest-cov

```bash
# Rapport terminal avec lignes manquantes
pytest --cov=src --cov-report=term-missing

# Rapport HTML
pytest --cov=src --cov-report=html

# Rapport XML (pour CI/CD)
pytest --cov=src --cov-report=xml

# Échouer si couverture < 80%
pytest --cov=src --cov-fail-under=80

# Couverture de branches
pytest --cov=src --cov-branch

# Plusieurs formats
pytest --cov=src --cov-report=html --cov-report=term
```

## 4. Améliorer la couverture

Ajoutons des tests pour atteindre 100% de couverture.

In [ ]:
%%writefile test_math_utils_complet.py
"""Tests complets pour math_utils (100% coverage)."""
import pytest
from math_utils import absolute, max_de_trois, factorielle, diviser_ou_zero

# Tests pour absolute
def test_absolute_positif():
    assert absolute(5) == 5
    assert absolute(0) == 0

def test_absolute_negatif():
    """✅ Nouveau : test de la branche négative."""
    assert absolute(-5) == 5
    assert absolute(-100) == 100

# Tests pour max_de_trois
def test_max_de_trois_a_max():
    """Test quand a est le maximum."""
    assert max_de_trois(3, 2, 1) == 3
    assert max_de_trois(5, 5, 3) == 5

def test_max_de_trois_b_max():
    """✅ Nouveau : test quand b est le maximum."""
    assert max_de_trois(1, 3, 2) == 3
    assert max_de_trois(3, 5, 5) == 5

def test_max_de_trois_c_max():
    """✅ Nouveau : test quand c est le maximum."""
    assert max_de_trois(1, 2, 3) == 3
    assert max_de_trois(5, 3, 5) == 5

# Tests pour factorielle
def test_factorielle_normal():
    assert factorielle(0) == 1
    assert factorielle(1) == 1
    assert factorielle(5) == 120

def test_factorielle_negatif():
    """✅ Nouveau : test de l'exception."""
    with pytest.raises(ValueError, match="positif"):
        factorielle(-1)

# Tests pour diviser_ou_zero
def test_diviser_ou_zero_normal():
    """✅ Nouveau : test de la fonction oubliée."""
    assert diviser_ou_zero(10, 2) == 5
    assert diviser_ou_zero(7, 2) == 3.5

def test_diviser_ou_zero_zero():
    """✅ Nouveau : test de la division par zéro."""
    assert diviser_ou_zero(10, 0) == 0

In [ ]:
# Vérifier la couverture améliorée
!pytest test_math_utils_complet.py --cov=math_utils --cov-report=term-missing --cov-branch

## 5. Branch Coverage vs Line Coverage

La **couverture de branches** est plus stricte que la couverture de lignes.

In [ ]:
%%writefile branch_example.py
"""Exemple de branch coverage."""

def verifier_age(age):
    """Vérifie si l'âge est valide."""
    if age < 0:
        return "invalide"
    elif age < 18:
        return "mineur"
    else:
        return "majeur"

In [ ]:
%%writefile test_branch_incomplete.py
"""Test incomplet (100% line coverage, mais pas branch coverage)."""
from branch_example import verifier_age

def test_age_majeur():
    """Teste seulement le cas majeur."""
    assert verifier_age(25) == "majeur"
    # Toutes les lignes sont exécutées, mais pas toutes les branches !

In [ ]:
# Line coverage : 100% (toutes les lignes sont exécutées)
!pytest test_branch_incomplete.py --cov=branch_example --cov-report=term-missing

In [ ]:
# Branch coverage : incomplet (branches if/elif non testées)
!pytest test_branch_incomplete.py --cov=branch_example --cov-report=term-missing --cov-branch

In [ ]:
%%writefile test_branch_complet.py
"""Test complet (100% branch coverage)."""
import pytest
from branch_example import verifier_age

@pytest.mark.parametrize("age,attendu", [
    (-1, "invalide"),   # Branche 1
    (10, "mineur"),     # Branche 2
    (25, "majeur"),     # Branche 3
])
def test_age_toutes_branches(age, attendu):
    """Teste toutes les branches."""
    assert verifier_age(age) == attendu

In [ ]:
# Maintenant : 100% branch coverage
!pytest test_branch_complet.py --cov=branch_example --cov-report=term-missing --cov-branch

## 6. Configuration de coverage dans pyproject.toml

In [ ]:
%%writefile pyproject_coverage.toml
[tool.coverage.run]
# Code source à mesurer
source = ["src"]

# Mesurer les branches
branch = true

# Fichiers à omettre
omit = [
    "*/tests/*",
    "*/test_*.py",
    "*/__init__.py",
    "*/migrations/*",
]

[tool.coverage.report]
# Seuil de couverture minimum
fail_under = 80

# Précision (2 décimales)
precision = 2

# Afficher les lignes manquantes
show_missing = true

# Ne pas afficher les fichiers à 100%
skip_covered = false

# Exclure certaines lignes
exclude_lines = [
    "pragma: no cover",
    "def __repr__",
    "raise AssertionError",
    "raise NotImplementedError",
    "if __name__ == .__main__.:",
    "if TYPE_CHECKING:",
    "@abstractmethod",
]

[tool.coverage.html]
directory = "htmlcov"

[tool.coverage.xml]
output = "coverage.xml"

## 7. Objectif de couverture : 80% ? 100% ?

### Le débat sur la couverture

| Objectif | Avantages | Inconvénients |
|----------|-----------|---------------|
| **60-70%** | Réaliste pour projets legacy | Beaucoup de code non testé |
| **80%** | ✅ **Recommandé** : bon équilibre | Nécessite de la discipline |
| **90%+** | Très haute confiance | Peut être coûteux en temps |
| **100%** | Couverture complète | Souvent impossible/inutile |

### Règles d'or

1. **100% de couverture ≠ 0 bugs**
   - La couverture mesure ce qui est *exécuté*, pas ce qui est *correct*

2. **Viser 80% est un bon objectif**
   - Les 20% restants sont souvent du code difficile à tester (UI, I/O, etc.)

3. **Privilégier la qualité sur la quantité**
   - Mieux vaut 60% de tests pertinents que 100% de tests bidons

4. **Code critique = 100% de couverture**
   - Logique métier, calculs financiers, sécurité → tester à fond

## 8. Tox : Tester sur plusieurs versions Python

**Tox** automatise les tests sur différents environnements Python.

In [ ]:
!pip install tox -q

In [ ]:
%%writefile tox.ini
[tox]
# Versions Python à tester
envlist = py38,py39,py310,py311,lint
isolated_build = True

[testenv]
# Dépendances pour les tests
deps =
    pytest
    pytest-cov
    requests

# Commandes à exécuter
commands =
    pytest --cov=math_utils --cov-report=term-missing {posargs}

[testenv:lint]
# Environnement pour le linting
deps =
    ruff
    mypy
commands =
    ruff check .
    mypy .

[testenv:coverage]
# Environnement pour générer le rapport HTML
deps =
    pytest
    pytest-cov
commands =
    pytest --cov=math_utils --cov-report=html --cov-report=term

### Utilisation de tox

```bash
# Exécuter tous les environnements
tox

# Exécuter un environnement spécifique
tox -e py310

# Exécuter le linting
tox -e lint

# Recréer les environnements
tox -r

# Passer des arguments à pytest
tox -e py310 -- -v -k test_specific
```

## 9. Nox : Alternative moderne à tox

**Nox** est comme tox, mais configuré en Python (pas en INI).

In [ ]:
!pip install nox -q

In [ ]:
%%writefile noxfile.py
"""Configuration Nox pour automatisation des tests."""
import nox

# Versions Python à tester
@nox.session(python=["3.8", "3.9", "3.10", "3.11"])
def tests(session):
    """Exécute les tests sur plusieurs versions Python."""
    session.install("pytest", "pytest-cov", "requests")
    session.run(
        "pytest",
        "--cov=math_utils",
        "--cov-report=term-missing",
        *session.posargs,
    )

@nox.session
def lint(session):
    """Exécute le linting."""
    session.install("ruff", "mypy")
    session.run("ruff", "check", ".")
    session.run("mypy", ".")

@nox.session
def coverage(session):
    """Génère le rapport de couverture HTML."""
    session.install("pytest", "pytest-cov")
    session.run(
        "pytest",
        "--cov=math_utils",
        "--cov-report=html",
        "--cov-report=term",
    )
    session.log("Rapport HTML généré dans htmlcov/index.html")

@nox.session
def format_code(session):
    """Formate le code."""
    session.install("ruff")
    session.run("ruff", "format", ".")

### Utilisation de nox

```bash
# Lister les sessions disponibles
nox --list

# Exécuter toutes les sessions
nox

# Exécuter une session spécifique
nox -s tests

# Exécuter sur Python 3.10 uniquement
nox -s tests-3.10

# Passer des arguments
nox -s tests -- -v -k test_specific

# Réutiliser l'environnement (plus rapide)
nox -s tests -r
```

## 10. Makefile pour automatiser les commandes

In [ ]:
%%writefile Makefile
.PHONY: test coverage lint format clean help

help:  ## Affiche cette aide
	@grep -E '^[a-zA-Z_-]+:.*?## .*$$' $(MAKEFILE_LIST) | sort | awk 'BEGIN {FS = ":.*?## "}; {printf "\033[36m%-20s\033[0m %s\n", $$1, $$2}'

test:  ## Exécute les tests
	pytest -v

coverage:  ## Génère le rapport de couverture
	pytest --cov=math_utils --cov-report=html --cov-report=term-missing
	@echo "Rapport HTML : htmlcov/index.html"

lint:  ## Vérifie le style de code
	ruff check .
	mypy .

format:  ## Formate le code
	ruff format .

clean:  ## Nettoie les fichiers générés
	rm -rf .pytest_cache .coverage htmlcov __pycache__ .mypy_cache .tox .nox
	find . -type d -name "*.egg-info" -exec rm -rf {} +
	find . -type f -name "*.pyc" -delete

install:  ## Installe les dépendances
	pip install -e ".[dev]"

all: format lint test coverage  ## Exécute tout

Utilisation :

```bash
make help      # Affiche l'aide
make test      # Exécute les tests
make coverage  # Génère le rapport
make lint      # Vérifie le style
make format    # Formate le code
make all       # Fait tout
```

## Pièges courants

### 1. Couverture 100% != code sans bugs

```python
# ❌ MAUVAIS : 100% de couverture, mais test inutile
def diviser(a, b):
    return a / b

def test_diviser():
    diviser(10, 2)  # Pas d'assertion !
    # 100% de couverture, mais le test ne vérifie rien

# ✅ BON : Test avec assertions
def test_diviser():
    assert diviser(10, 2) == 5
    with pytest.raises(ZeroDivisionError):
        diviser(10, 0)
```

### 2. Tester les branches, pas seulement les lignes

```python
# ❌ MAUVAIS : Ligne couverte, mais pas toutes les branches
def fonction(x):
    return "positif" if x > 0 else "négatif ou zéro"

def test_fonction():
    fonction(5)  # Seulement la branche True

# ✅ BON : Toutes les branches
@pytest.mark.parametrize("x,attendu", [
    (5, "positif"),
    (-5, "négatif ou zéro"),
    (0, "négatif ou zéro"),
])
def test_fonction(x, attendu):
    assert fonction(x) == attendu
```

### 3. Tests lents qui bloquent le développement

```python
# ❌ MAUVAIS : Tests lents exécutés à chaque fois
def test_integration_complete():
    time.sleep(10)  # 10 secondes
    # ...

# ✅ BON : Marquer les tests lents
@pytest.mark.slow
def test_integration_complete():
    time.sleep(10)
    # ...

# Exécuter sans les tests lents :
# pytest -m "not slow"
```

### 4. Oublier de tester sur plusieurs versions Python

```python
# Code qui fonctionne en Python 3.10+
def fonction(data: list[int]) -> int:  # list[int] nécessite 3.9+
    return sum(data)

# ✅ Tester avec tox/nox sur 3.8, 3.9, 3.10, 3.11
```

## Mini-Exercices

### Exercice 1 : Ajouter coverage à un projet

Créez un module et des tests, puis mesurez la couverture.

In [ ]:
%%writefile string_operations.py
def inverser(texte):
    """Inverse un texte."""
    return texte[::-1]

def compter_voyelles(texte):
    """Compte les voyelles dans un texte."""
    voyelles = "aeiouyAEIOUY"
    return sum(1 for c in texte if c in voyelles)

def premier_mot(phrase):
    """Retourne le premier mot d'une phrase."""
    if not phrase:
        return ""
    return phrase.split()[0] if phrase.split() else ""

In [ ]:
# TODO: Écrivez des tests et mesurez la couverture
# Objectif: 100% de couverture


### Exercice 2 : Configurer tox ou nox

Créez un fichier tox.ini ou noxfile.py pour votre projet.

In [ ]:
# Votre configuration ici


### Exercice 3 : Identifier du code non couvert

Trouvez et corrigez les tests manquants.

In [ ]:
%%writefile exercice3.py
def classifier_nombre(n):
    """Classifie un nombre."""
    if n < 0:
        return "negatif"
    elif n == 0:
        return "zero"
    elif n < 10:
        return "petit_positif"
    else:
        return "grand_positif"

In [ ]:
%%writefile test_exercice3_incomplet.py
from exercice3 import classifier_nombre

def test_classifier_zero():
    assert classifier_nombre(0) == "zero"

def test_classifier_grand_positif():
    assert classifier_nombre(100) == "grand_positif"

# TODO: Ajoutez les tests manquants pour atteindre 100% de couverture

## Solutions

### Solution Exercice 1

In [ ]:
%%writefile test_string_operations_solution.py
import pytest
from string_operations import inverser, compter_voyelles, premier_mot

def test_inverser():
    assert inverser("hello") == "olleh"
    assert inverser("") == ""
    assert inverser("a") == "a"

def test_compter_voyelles():
    assert compter_voyelles("hello") == 2
    assert compter_voyelles("aeiou") == 5
    assert compter_voyelles("xyz") == 1  # y
    assert compter_voyelles("") == 0

def test_premier_mot():
    assert premier_mot("hello world") == "hello"
    assert premier_mot("python") == "python"
    assert premier_mot("") == ""
    assert premier_mot("   ") == ""

In [ ]:
!pytest test_string_operations_solution.py --cov=string_operations --cov-report=term-missing --cov-branch

### Solution Exercice 2

In [ ]:
%%writefile noxfile_solution.py
import nox

@nox.session(python=["3.9", "3.10", "3.11"])
def tests(session):
    """Exécute les tests."""
    session.install("pytest", "pytest-cov")
    session.run(
        "pytest",
        "--cov=string_operations",
        "--cov-report=term-missing",
        "--cov-branch",
        "--cov-fail-under=80",
    )

@nox.session
def lint(session):
    """Linting."""
    session.install("ruff")
    session.run("ruff", "check", ".")

@nox.session
def coverage_html(session):
    """Génère le rapport HTML."""
    session.install("pytest", "pytest-cov")
    session.run(
        "pytest",
        "--cov=string_operations",
        "--cov-report=html",
    )

### Solution Exercice 3

In [ ]:
%%writefile test_exercice3_solution.py
import pytest
from exercice3 import classifier_nombre

@pytest.mark.parametrize("n,attendu", [
    (-5, "negatif"),
    (-1, "negatif"),
    (0, "zero"),
    (1, "petit_positif"),
    (9, "petit_positif"),
    (10, "grand_positif"),
    (100, "grand_positif"),
])
def test_classifier_nombre(n, attendu):
    """Test complet avec toutes les branches."""
    assert classifier_nombre(n) == attendu

In [ ]:
!pytest test_exercice3_solution.py --cov=exercice3 --cov-report=term-missing --cov-branch